### Check if there are duplicates row and drop it

In [0]:
from pyspark.sql.functions import count

In [0]:
def drop_duplicates(df):
    duplicates = df.groupBy(df.columns).count().filter("count > 1")
    if duplicates.count() > 0:
        df = df.dropDuplicates()
        return 'Duplicates dropped'
    else:
        return 'No duplicates'